# Settings

In [37]:
IN_FILENAME = "../data/processed/calculated.xlsx"
OUT_FILENAME = "../data/product/filtered.xlsx"

SEPARATOR = "\n"

### adjustments

In [38]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [39]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

### constants imports

In [40]:
with open("../constans/columns.json", "r") as file: columns, years = json.load(file).values()

columns = namedtuple('Columns', ' '.join(columns.keys()))(**columns)
years = namedtuple('Years', re.sub(r'[\[\],\']','', str([*map(lambda y: f'y{y}', years)])))(*years)

### tools

In [41]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

# Main

In [42]:
filter_columns = [
    columns.nombre,

   *[YYYY(columns.TD_div_TA_YYYY, year) for year in years],
   *[YYYY(columns.WC_div_TA_YYYY, year) for year in years],
   *[YYYY(columns.EBITDA_div_TA_YYYY, year) for year in years],
   *[YYYY(columns.lnTA_YYYY, year) for year in years],
   columns.isFamiliar
]

In [43]:
data = pd.read_excel(IN_FILENAME)

In [44]:
df = data[filter_columns]

In [45]:
df.head()

,Nombre,TD/TA_2021,TD/TA_2022,TD/TA_2023,WC/TA_2021,WC/TA_2022,WC/TA_2023,EBITDA/TA_2021,EBITDA/TA_2022,EBITDA/TA_2023,ln(TA)_2021,ln(TA)_2022,ln(TA)_2023,isFamiliar
0,FELIX SOLIS SOCIEDAD LIMITADA,28.881117,29.229926,32.205115,5.123154e+06,4.178322e+06,5.167578e+06,6.619700e+06,2.545090e+06,7.408183e+06,0.358703,0.336391,0.424802,unknown
1,CODORNIU SA,0.745956,0.985328,0.661631,8.501304e+07,6.806911e+07,1.023132e+08,1.611695e+07,1.099225e+07,2.269676e+07,-0.033204,0.021107,-0.155104,no
2,MIGUEL TORRES SA,1.832514,2.061384,2.266592,9.451695e+07,8.441475e+07,7.487256e+07,8.732763e+06,7.709204e+06,-3.646480e+06,-0.170886,-0.253326,-0.215693,unknown
3,GARCIA CARRION 1890 SL,3.650187,3.568547,8.457106,1.063484e+07,2.095649e+08,1.732161e+08,3.527727e+06,1.389730e+08,1.977129e+08,-0.104308,-1.393215,-1.703137,yes
4,"BARON DE LEY, S.A.",110.334803,9.072074,3.148995,8.313455e+07,1.875481e+08,9.133677e+07,1.642012e+07,6.938583e+07,3.007278e+07,0.333328,-0.655692,0.247142,no


In [46]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")